In [2]:
import timeit
import os
import pickle
import json
#os.environ["THEANO_FLAGS"]="floatX=float32"
#os.environ["THEANO_FLAGS"]="floatX=float32,dnn.enabled=False,cxx=icpc,
#                            device=gpu0,nvcc.compiler_bindir=icpc,
#                            gcc.cxxflags=-march=native"
import numpy
from hyperopt import fmin, tpe, hp, Trials
from training import objective_train_model
from predict import eval_train_model
from optparse import OptionParser

### Look for the best set of parameters for the bi-LSTM, after trying 100 random combinations out of possible 16K in the search space  using objective_train_model and hyperopt TPE search.

In [ ]:
def parameter_search(ntrials, hyperopt_seed, is_test=False):
    start = timeit.default_timer()
    trials = Trials()

    if is_test:
        # Unrealistic values of parameters; for testing use only
        search_space= { 'num_dense_layers': hp.choice('nlayers',[1, 2, 3, 4]),
                        'num_dense_units': hp.choice('num_dense', [2, 3, 4, 5]),
                        'num_epochs': hp.choice('num_epochs',  [3, 5, 7, 10]),
                        'num_lstm_units': hp.choice('num_lstm_units',  [2, 3]),
                        'num_lstm_layers': hp.choice('num_lstm_layers', [1, 2]),
                        'learn_rate': hp.choice('learn_rate',[1e-4, 3e-4, 1e-3]),
                        'mb_size': hp.choice('mb_size', [32, 64, 100, 120]),
                        'l2reg': hp.choice('l2reg', [0.0, 1e-4, 3e-4, 1e-3]),
                        'rng_seed': hp.choice('rng_seed', [364])
    }
    else:
        # Parameter values tested for SemEval-2019 Task 8
        search_space= { 'num_dense_layers': hp.choice('nlayers',[1,2,3,4]),
                        'num_dense_units': hp.choice('num_dense', [100, 200, 300,
                                                                   400, 500]),
                        'num_epochs': hp.choice('num_epochs',  [30, 50, 70, 100]),
                        'num_lstm_units': hp.choice('num_lstm_units',  [100, 200,
                                                                        300]),
                        'num_lstm_layers': hp.choice('num_lstm_layers', [1,2]),
                        'learn_rate': hp.choice('learn_rate',[1e-4, 3e-4, 1e-3]),
                        'mb_size': hp.choice('mb_size', [32, 64, 100, 120]),
                        'l2reg': hp.choice('l2reg', [0.0, 1e-4, 3e-4, 1e-3]),
                        'rng_seed': hp.choice('rng_seed', [364])
        }

    # Find the best set of hyperparameters (with TPE algorithm)
    best = fmin(objective_train_model,
                space=search_space,
                algo=tpe.suggest,
                max_evals=ntrials,
                trials=trials,
                rstate=numpy.random.RandomState(hyperopt_seed))
    
    params = trials.best_trial['result']['Params']

    print "\nBest hyperparameters are:"
    for hp_name, hp_value in params.items():
        print "%-20s%-.5g" % (hp_name, hp_value)
        
    out_path = 'output'
    if not os.path.exists(out_path):
            os.makedirs(out_path)
    
    f = open(os.path.join(out_path, 'trials.txt'), "w+")
    pickle.dump(trials, f)
    f.close()
    
    f = open(os.path.join(out_path, 'bestparams.txt'), "w+")
    pickle.dump(params, f)
    f.close()
    
    print "\nSaved trials and optimal hyperparameters to trials.txt and bestparams.txt"
    
    stop = timeit.default_timer()
    print "Total time: ", stop - start, "\n"
    
    return params

In [ ]:
parser = OptionParser()
parser.add_option(
        '--search', dest='psearch', default=False,
        help='Whether parameter search should be done: default=%default')
parser.add_option('--ntrials', dest='ntrials', default=10,
                  help='Number of trials: default=%default')
parser.add_option('--hseed', dest='hseed', default=None,
        help="Value of the rng seed passed to hyperopt's fmin function: default=%default")
parser.add_option(
        '--params', dest='params_file', default='output/bestparams_semeval2017.txt',
        help='Location of parameter file: default=%default')
parser.add_option(
        '--test', dest='is_test', default=False,
        help='Run with test parameters: default=%default')



(options, args) = parser.parse_args()
psearch = options.psearch
ntrials = int(options.ntrials)
hyperopt_seed = options.hseed
if hyperopt_seed is not None:
    hyperopt_seed = int(hyperopt_seed)
is_test = options.is_test
params_file = options.params_file
params = parameter_search(ntrials, hyperopt_seed, is_test)